# Part 1: Credit Card Statement Reconciliation Process

In [1]:
import math
import numpy as np
import pandas as pd
import re

# 1. Understanfing the data

In [2]:
# Import the data 
statements_df=pd.read_csv('data/Card Data (Data Scientist) - Loan Tape corrected.csv')

![](assets/basic_ERD.png)

# 2. Prepare the data

Let's create a new dataframe where currency data is treated as floats and dates are correctly parsed

In [3]:
# Convert currency string values to floats
def apply_currency_check(series: pd.Series, currency_pattern: re.Pattern) -> pd.Series:
    def process_value(x: str) -> [float, pd.NA]:
        if pd.notna(x) and currency_pattern.match(x):
            formated_number = float(
                currency_pattern.match(x).group().replace("$", "").replace(",", "")
            )
        else:
            formated_number = pd.NA
        return formated_number

    return series.apply(process_value)


# time and currency text patterns
date_pattern = re.compile(r"\d{4}-\d{2}-\d{2}$")
datetime_pattern = re.compile(r"\d{4}-\d{2}-\d{2} \d{1,2}:\d{1,2}:\d{1,2}")
currency_pattern = re.compile(r"(-)?\$\d{1,3}(,\d{3})*(\.\d{1,2})?")

# Creates a new dataframe with the data correctly parsed
statements_formatted_df = pd.DataFrame()
for column, series in statements_df.items():
    # Set the headers to lower case and add low dashes in between
    formated_header = f"{column.lower().replace(' ', '_')}"
    dates_check = series.str.match(date_pattern, na=False)
    datetime_check = series.str.match(datetime_pattern, na=False)
    currency_check = series.apply(lambda x: pd.notna(x) and currency_pattern.match(x))

    if dates_check.any():
        statements_formatted_df[formated_header] = pd.to_datetime(
            series, errors="coerce", format="%Y-%m-%d"
        )
    elif datetime_check.any():
        statements_formatted_df[formated_header] = pd.to_datetime(
            series, errors="coerce", format="%Y-%m-%d %H:%M:%S"
        )
    elif currency_check.any():
        statements_formatted_df[formated_header] = apply_currency_check(
            series, currency_pattern
        )
    else:
        statements_formatted_df[formated_header] = series

# Double checks if the data is correctly sorted for the credit card reconciliation process
sort_keys = ["business_id", "statement_start_date"]
statements_formatted_df.sort_values(by=sort_keys, ascending=True, inplace=True)

# 3. Payment date reconcilation

The rules for reconciling a payment date are as follows:
* Examine the balance of the previous transaction. If it is positive and lower than the current statement payment, assign the payment date to the group of statements.
* If both the previous outstanding balance and the current one are lower or equal to zero, no incurred debt exists. Therefore, the payment date is null.
* Mark a group of statement payment dates as 'pending' if there are no more statements to assess the payment_date.
The only missing data needed to accomplish this process is the outstanding balance prior to the first statement. This can be determined by accounting for the first statement's total payments, total purchases, fees, and refunds. For the most part, the result should be zero.

In [4]:
# Shift the outstanding balance down to assess if the banace was paid in full
statements_formatted_df["previous_outstanding_balance"] = (
    statements_formatted_df.groupby("business_id")["outstanding_balance"]
    .shift()
)

In [5]:
# Estimate the last statement outstanding balance by subtracting the aggregated sum from the first outstanding balance
def first_previous_outstanding_balance(statements_formatted_df: pd.DataFrame, out_column:str):
    # Do it for each business
    for _, group in statements_formatted_df.groupby("business_id"):
        index=group.index[0]
        first_row = group.iloc[0]
        # Get values
        domestic_amount=first_row["domestic_amount"]
        international_amount=first_row["international_amount"]
        fee_amount=first_row["fee_amount"]
        total_refunds=first_row["total_refunds"]
        total_payment_collected=first_row["total_payment_collected"]
        outstanding_balance=first_row["outstanding_balance"]
        current_value=first_row[out_column]
        # Compute the last period outstanding balance by subtracting the aggregated of purchases, fees, paymens and refunds. 
        value_to_input = round(outstanding_balance-(
            domestic_amount + international_amount + fee_amount - total_refunds-total_payment_collected), 2
        )
        if pd.isna(current_value):
            statements_formatted_df.loc[index, out_column]=value_to_input
        else:
            statements_formatted_df.loc[index, out_column]+=value_to_input

first_previous_outstanding_balance(statements_formatted_df,"previous_outstanding_balance")        

In [6]:
# Reconcile the payments by assessing when the bebt, if any, was paid in full
def reconcile_payment_dates(statements_formatted_df: pd.DataFrame):
    for _, group in statements_formatted_df.groupby("business_id"):
        previous_outstandings = group["previous_outstanding_balance"]
        outstanding_balances = group["outstanding_balance"]
        total_payments = group["total_payment_collected"]
        payment_dates = group["payment_date"]
        payment_dates_reconcile = []
        reconciled_index = -1
        # Loop through the rows of each statement
        for item, (
            prev_outstanding,
            outstanding_balance,
            total_payment,
            payment_date,
        ) in enumerate(
            zip(
                previous_outstandings,
                outstanding_balances,
                total_payments,
                payment_dates,
            )
        ):
            items_to_add = item - reconciled_index if item != 0 else 1
            # If there is a current debt, it has been paid in full and there are no positive balances, then impute the last payment date
            if prev_outstanding <= total_payment and prev_outstanding > 0:
                payment_dates_reconcile.extend([payment_date] * items_to_add)
                reconciled_index = item
            # Mark as NA if no debt was incurred.
            elif (
                prev_outstanding <= 0
                and item != len(previous_outstandings) - 1
                and outstanding_balance <= 0
            ):
                payment_dates_reconcile.extend([pd.NaT] * items_to_add)
                reconciled_index = item
            # Mark as pending if no payments have been made reaching the end of the list
            elif item == (len(previous_outstandings) - 1):
                payment_dates_reconcile.extend(["outstanding"] * items_to_add)
                reconciled_index = item
            # Any other scenario, keep looking for the payment
            else:
                continue
        # Create the column in the given dataframe
        statements_formatted_df.loc[
            group.index, "payment_date_reconcile"
        ] = payment_dates_reconcile


reconcile_payment_dates(statements_formatted_df)

# 4. Identify missed payments

Business logic:
* A payment is considered missed if there is a difference of more than 30 days between the payment_date and statement_end_date.
* Negative values may occur due to prepayment errors.

In [7]:
# Function to compute the number of days between the payment date and the statement end date
def compute_days_past_due(payment_date, statement_end_date):
    if pd.isna(payment_date) or payment_date == "outstanding":
        return pd.NA
    else:
        return (payment_date - statement_end_date).days


# Define a function to check missed payments
def check_missed_payments(past_due_days):
    return pd.notna(past_due_days) and past_due_days > 30 and past_due_days != "outstanding"

In [8]:
# Subtract the reconciled date with the statement end date
statements_formatted_df['days_past_due']=statements_formatted_df.apply(lambda row: compute_days_past_due(row['payment_date_reconcile'], row['statement_end_date']), axis=1)
# Apply the function to create the 'missed_payments' column
statements_formatted_df['missed_payments'] = statements_formatted_df['days_past_due'].apply(check_missed_payments)

# 5. Reconcile transactions and payments

The idea is to sum payments up to the point that the outstanding balance was paid in full, creating a new column named 'total_payment_reconcile.' This will be later useful for forecasting purposes.

In [9]:
# Aggregate the payments and assign it to the corresponding transaction
def reconcile_total_payments(statements_formatted_df: pd.DataFrame, out_column: str):
    statements_formatted_df[out_column] = 0.0
    for _, group in statements_formatted_df.groupby("business_id"):
        # Get a list of the unique payment dates that are in 'payment_date_reconcile'
        payment_dates = pd.to_datetime(
            group["payment_date_reconcile"], errors="coerce", format="%Y-%m-%d %H:%M:%S"
        )
        payment_dates = payment_dates.dropna()
        # Assign paymets, if there are some
        if not payment_dates.empty:
            transaction_index = ~payment_dates.duplicated(keep="last")
            filtered_payment_dates = payment_dates[transaction_index]
            last_day = pd.NaT
            # Loop though the payments and add the totals
            for index, date in filtered_payment_dates.items():
                date_filter = (
                    group["payment_date"] <= date
                    if pd.isna(last_day)
                    else group["payment_date"].between(
                        last_day, date, inclusive="right"
                    )
                )
                total_payment = group.loc[date_filter, "total_payment_collected"].sum()
                statements_formatted_df.loc[index, out_column] = total_payment
                last_day = date

reconcile_total_payments(statements_formatted_df, "total_payment_reconcile")

I've also cross check that the outstanding balances were computed correctly, untill i realize what was really asked.

In [10]:
# Compute each statement balance
statements_formatted_df["statement_balance"] = (
    statements_formatted_df["domestic_amount"]
    + statements_formatted_df["international_amount"]
    + statements_formatted_df["fee_amount"]
    - statements_formatted_df["total_refunds"]
    - statements_formatted_df["total_payment_collected"]
)
# Add to the first element the last term outstanding balance
first_previous_outstanding_balance(statements_formatted_df, "statement_balance")
# Reconcile the outstanding balance by doing the comulative sum over the groups
statements_formatted_df["outstanding_balance_recon"]=statements_formatted_df.groupby('business_id')["statement_balance"].cumsum()
statements_formatted_df["outstanding_balance_recon"]=statements_formatted_df["outstanding_balance_recon"].round(2)
statements_formatted_df["outstanding_balance_error"]=(statements_formatted_df["outstanding_balance_recon"]!=statements_formatted_df["outstanding_balance"])

# 6. Summary report

In [11]:
# Compute the mean but exclude pd.NA and negative numbers
def custom_mean(series:pd.Series)-> float:
    # Replace negative values with zeros
    series = series.apply(lambda x: max(0, x) if pd.notna(x) else x)
    # Exclude pd.NA
    positive_values = series[~series.isna()]
    # Calculate the mean and round up
    average = math.ceil(positive_values.mean()) if not positive_values.empty else None
    return average

In [12]:
# 1.Business ID
business_formatted_df = pd.DataFrame(
    {"business_id": statements_formatted_df["business_id"].unique()}
)
# 2.Overdue payments count
business_formatted_df["overdue_payments"] = (
    statements_formatted_df.groupby("business_id")["missed_payments"]
    .sum()
    .reset_index(drop=True)
)
# 3. Count the number of outstanding payments
outstanding_payments_series=statements_formatted_df[
        statements_formatted_df["payment_date_reconcile"] == "outstanding"
    ].groupby("business_id")["payment_date_reconcile"].value_counts().unstack()["outstanding"].rename("outstanding_payments")
business_formatted_df=pd.merge(business_formatted_df,outstanding_payments_series, how='left',left_on='business_id', right_index=True)
# 3.Average days to pay
business_formatted_df["average_days_payment"] = (
    statements_formatted_df.groupby("business_id")["days_past_due"]
    .apply(custom_mean)
    .reset_index(drop=True)
)

In [13]:
# 5. Days without receiving a payment of an active outstading balance
def get_days_without_payment(statements_formatted_df: pd.DataFrame, business_formatted_df:pd.DataFrame):
    # Filter only rows with 'outstanding' payments
    outstanding_payments_df = statements_formatted_df[statements_formatted_df["payment_date_reconcile"] == 'outstanding']
    # Group by business_id and aggregate the first and last statement_end_date
    agg_result = outstanding_payments_df.groupby("business_id")["statement_end_date"].agg(['first', 'last'])
    # Calculate the days without payment using vectorized operations
    agg_result['outstanding_payment_days'] =agg_result.apply(lambda x: (x['last'] - x['first']).days if (x['last'] != x['first']) else np.nan,axis=1)
    agg_result = agg_result.reset_index()[["business_id", "outstanding_payment_days"]]
    # Merge with business_formatted_df
    merged_df = pd.merge(business_formatted_df, agg_result, how='left', left_on='business_id', right_on='business_id')
    return merged_df

business_formatted_df=get_days_without_payment(statements_formatted_df,business_formatted_df)

In [14]:
# 8. Average credit utilization
outstandings_with_limit =statements_formatted_df ['total_credit_limit'] != 0
# Calculate the division result for each row
statements_formatted_df['avg_credit_utilization'] = statements_formatted_df.loc[outstandings_with_limit,'outstanding_balance_recon']*100 / statements_formatted_df.loc[outstandings_with_limit,'total_credit_limit']
# Calculate the average division result for each group
result_df = statements_formatted_df.groupby('business_id')['avg_credit_utilization'].mean().round(2).reset_index()
result_df['avg_credit_utilization'] = np.maximum(0, result_df['avg_credit_utilization'])
business_formatted_df=pd.merge(business_formatted_df,result_df, how='left',on='business_id')

In [15]:
# 7.Current outstading_balances
business_formatted_df["current_outstanding_balance"] = (
    statements_formatted_df.groupby("business_id")["outstanding_balance_recon"]
    .last()
    .reset_index(drop=True)
)
# Sort the data according to the relevance of the KPIs
sort_keys=['outstanding_payment_days','average_days_payment','overdue_payments']
business_formatted_df.sort_values(by=sort_keys, ascending=False, inplace=True)

In [16]:
business_formatted_df

,business_id,overdue_payments,outstanding_payments,average_days_payment,outstanding_payment_days,avg_credit_utilization,current_outstanding_balance
9,cl8khb2qs11621mx5wkbus530,0,29.0,NaN,196.0,51.42,71663.13
1,cl81puo9y23001kwys64o0gfq,1,7.0,10.0,134.0,16.24,7371.30
8,cl8jb9bog03311lx6vzqbhsrd,0,19.0,2.0,126.0,7.58,218.35
0,cl81p742620381lycqw0xl7sf,1,14.0,9.0,91.0,23.40,4442.14
19,cl9t40tte000by81lrq7xbe5b,0,4.0,1.0,89.0,52.32,79551.83
37,cldbm1m9p000jx91lyp9p0xi8,0,4.0,NaN,89.0,54.07,154723.51
22,clau4nua3000bx31k9eb3kanl,0,3.0,NaN,68.0,33.85,9666.93
16,cl9j0txv00000yk1kgjbjnphe,0,3.0,1.0,61.0,14.97,4676.66
43,cldnwz523000dwu1kegcx699l,0,3.0,NaN,61.0,51.69,19640.73
24,clavkfksj000ex21l5fidbsay,0,2.0,1.0,30.0,17.63,11887.52


# 7. Save the data for part two

In [17]:
statements_formatted_df.to_csv("data/reconciled_card_data.csv", index=False)
business_formatted_df.to_csv("data/business_kpi.csv", index=False)